In [ ]:
!pip install transformers
# !pip install tensorflow

In [6]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Input, Dropout, Dense

In [31]:
# Import data
df = pd.read_csv('app_reviews_all_annotated2.csv')
df = df[['review', 'argument_cat', 'decision_cat']]

# Remove missing rows
df = df.dropna()

# Convert to numeric for bert
df['Argument'] = pd.Categorical(df['argument_cat'])
df['Decision'] = pd.Categorical(df['decision_cat'])
df['argument_cat'] = df['Argument'].cat.codes
df['decision_cat'] = df['Decision'].cat.codes

# Split into training and testing 
df, df_test = train_test_split(df, test_size = 0.2, stratify=df[['argument_cat']])

In [32]:
# BERT model
modelName = 'bert-base-uncased'
maxLen = 100
conf = BertConfig.from_pretrained(modelName)
conf.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = modelName, config = conf)

# Load transformer BERT model
transformerModel = TFBertModel.from_pretrained(modelName, config = conf)

bert = transformerModel.layers[0]
inputIds = Input(shape=(maxLen,), name='input_ids', dtype='int32')
inputs = {'input_ids': inputIds}

bertModel = bert(inputs)[1]
dropout = Dropout(conf.hidden_dropout_prob, name='pooled_output')
pooledOutput = dropout(bertModel, training=False)

arg = Dense(units=len(df.argument_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='argument')(pooledOutput)
dec = Dense(units=len(df.decision_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='decision')(pooledOutput)
outputs = {'argument': arg, 'decision': dec}

# Show model
model = Model(inputs=inputs, outputs=outputs, name='BERT_For_App_Review_Classification')
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "BERT_For_App_Review_Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
argument (Dense)                (None, 4)            3076        pooled_output[0][0]              
_________________________________________________________________

In [33]:
# Training
optimizer = Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
loss = {'argument': CategoricalCrossentropy(from_logits = True), 'decision': CategoricalCrossentropy(from_logits = True)}
metric = {'argument': CategoricalAccuracy('accuracy'), 'decision': CategoricalAccuracy('accuracy')}
model.compile(optimizer = optimizer, loss = loss, metrics = metric)

x = tokenizer(text=df.review.to_list(), add_special_tokens=True, max_length=maxLen, 
              truncation=True, padding=True, return_tensors='tf', return_token_type_ids = False,
              return_attention_mask = True, verbose = True)

history = model.fit(x={'input_ids': x['input_ids']}, y={'argument': to_categorical(df.argument_cat), 'decision': to_categorical(df.decision_cat)},
                    validation_split=0.2, batch_size=64, epochs=10)

Epoch 1/10
28/28 [==============================] - 55s 1s/step - loss: 1.9199 - argument_loss: 0.5166 - decision_loss: 1.4033 - argument_accuracy: 0.8332 - decision_accuracy: 0.4132 - val_loss: 1.6566 - val_argument_loss: 0.3390 - val_decision_loss: 1.3176 - val_argument_accuracy: 0.8691 - val_decision_accuracy: 0.4808
Epoch 2/10
28/28 [==============================] - 35s 1s/step - loss: 1.6145 - argument_loss: 0.2988 - decision_loss: 1.3157 - argument_accuracy: 0.8847 - decision_accuracy: 0.4698 - val_loss: 1.5432 - val_argument_loss: 0.3082 - val_decision_loss: 1.2350 - val_argument_accuracy: 0.8736 - val_decision_accuracy: 0.5056
Epoch 3/10
28/28 [==============================] - 35s 1s/step - loss: 1.4113 - argument_loss: 0.2729 - decision_loss: 1.1384 - argument_accuracy: 0.9073 - decision_accuracy: 0.5591 - val_loss: 1.2120 - val_argument_loss: 0.2788 - val_decision_loss: 0.9332 - val_argument_accuracy: 0.8871 - val_decision_accuracy: 0.6591
Epoch 4/10
28/28 [================

In [55]:
# Evaluation
testArg = to_categorical(df_test['argument_cat'], 4)
testDec = to_categorical(df_test['decision_cat'])
testReview = tokenizer(text=df_test['review'].to_list(), add_special_tokens=True,
                         max_length=maxLen, truncation=True, padding=True, 
                         return_tensors='tf', return_token_type_ids = False,
                         return_attention_mask = False, verbose = True)
modelEval = model.evaluate(x={'input_ids': testReview['input_ids']},
                                 y={'argument': testArg, 'decision': testDec})

18/18 [==============================] - 4s 193ms/step - loss: 0.7137 - argument_loss: 0.3119 - decision_loss: 0.4018 - argument_accuracy: 0.8861 - decision_accuracy: 0.8987
